In [1]:
import warnings
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'

# Suprimir advertencias FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def download_data(start, end, interval, ticker):
    return yf.download(ticker, start=start, end=end, interval=interval)

In [39]:
def calculate_signals(data, indicator):
    if indicator == 'Hold':
        hold_array = np.zeros_like(data['Close'], dtype = int)
        hold_array[0] = 1
        hold_array[-1] = -1
        data['Position'] = hold_array

    elif indicator == 'SMA':
        # Calcular medias móviles simples usando pandas_ta
        data['SMA_50'] = ta.sma(data['Close'], length=50)
        data['SMA_200'] = ta.sma(data['Close'], length=200)
        
        # Señales de cruce de medias móviles simples
        data['Signal_SMA'] = np.where(data['SMA_50'] > data['SMA_200'], 1, 0)
        data['Position'] = data['Signal_SMA'].diff()
        data['Position'] = np.where(data['Position'] == 1, 1, np.where(data['Position'] == -1, -1, 0))
        
    elif indicator == 'EMA':
        # Calcular medias móviles exponenciales usando pandas_ta
        data['EMA_50'] = ta.ema(data['Close'], length=50)
        data['EMA_200'] = ta.ema(data['Close'], length=200)
        
        # Señales de cruce de medias móviles exponenciales
        data['Signal_EMA'] = np.where(data['EMA_50'] > data['EMA_200'], 1, 0)
        data['Position'] = data['Signal_EMA'].diff()
        data['Position'] = np.where(data['Position'] == 1, 1, np.where(data['Position'] == -1, -1, 0))

    elif indicator == 'RSI':
        # Calcular RSI
        data['RSI'] = ta.rsi(data['Close'], length=14)
        
        # Señales de RSI
        data['RSI_Signal'] = 0
        data['RSI_Signal'] = np.where(data['RSI'] < 30, 1, data['RSI_Signal'])  # Señal de compra
        data['RSI_Signal'] = np.where(data['RSI'] > 70, -1, data['RSI_Signal'])  # Señal de venta
        data['Position'] = data['RSI_Signal']

    elif indicator == 'Bollinger':        
        # Calcular Bandas de Bollinger
        bbands_result = ta.bbands(data['Close'], length=20, std=2)

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Bollinger_Upper'] = bbands_result['BBU_20_2.0']
        data['Bollinger_Middle'] = bbands_result['BBM_20_2.0']
        data['Bollinger_Lower'] = bbands_result['BBL_20_2.0']

        # Opcionalmente, si necesitas otras columnas como el ancho y la desviación estándar
        data['Bollinger_width'] = bbands_result['BBB_20_2.0']
        data['Bollinger_std'] = bbands_result['BBP_20_2.0']
        
        # Señales de Bandas de Bollinger
        data['Bollinger_Signal'] = 0
        data['Bollinger_Signal'] = np.where(data['Close'] < data['Bollinger_Lower'], 1, data['Bollinger_Signal'])  # Señal de compra
        data['Bollinger_Signal'] = np.where(data['Close'] > data['Bollinger_Upper'], -1, data['Bollinger_Signal'])  # Señal de venta
        data['Position'] = data['Bollinger_Signal']

    elif indicator == 'Stochastic':
        # Calcular Oscilador Estocástico
        sto_result = ta.stoch(data['High'], data['Low'], data['Close'], k=14, d=3, smooth_k=3)
        
        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Stoch_K'] = sto_result['STOCHk_14_3_3']
        data['Stoch_D'] = sto_result['STOCHd_14_3_3']
        
        # Señales del Oscilador Estocástico
        data['Stoch_Signal'] = 0
        data['Stoch_Signal'] = np.where((data['Stoch_K'] < 20) & (data['Stoch_D'] < 20), 1, data['Stoch_Signal'])  # Señal de compra
        data['Stoch_Signal'] = np.where((data['Stoch_K'] > 80) & (data['Stoch_D'] > 80), -1, data['Stoch_Signal'])  # Señal de venta
        data['Position'] = data['Stoch_Signal']

    elif indicator == 'Volume':
        # Calcular Media Móvil de Volumen
        data['Volume_SMA_20'] = ta.sma(data['Volume'], length=20)
        
        # Señales de Volumen
        data['Volume_Signal'] = 0
        data['Volume_Signal'] = np.where(data['Volume'] > data['Volume_SMA_20'], 1, data['Volume_Signal'])  # Señal de compra
        data['Volume_Signal'] = np.where(data['Volume'] < data['Volume_SMA_20'], -1, data['Volume_Signal'])  # Señal de venta
        data['Position'] = data['Volume_Signal']
        
    elif indicator == 'EMA_RSI':
        # Calcular medias móviles exponenciales usando pandas_ta
        data['EMA_50'] = ta.ema(data['Close'], length=50)
        data['EMA_200'] = ta.ema(data['Close'], length=200)
        
        # Calcular RSI
        data['RSI'] = ta.rsi(data['Close'], length=14)
        
        # Señales de cruce de EMA y RSI
        data['EMA_RSI_Signal'] = 0
        data['EMA_RSI_Signal'] = np.where((data['EMA_50'] > data['EMA_200']) & (data['RSI'] < 30), 1, data['EMA_RSI_Signal'])  # Señal de compra
        data['EMA_RSI_Signal'] = np.where((data['EMA_50'] < data['EMA_200']) & (data['RSI'] > 70), -1, data['EMA_RSI_Signal'])  # Señal de venta
        data['Position'] = data['EMA_RSI_Signal']

    elif indicator == 'Ichimoku':
        # Calcular Ichimoku Kinko Hyo usando pandas_ta
        visible_period, forward_period = ta.ichimoku(data['High'], data['Low'], data['Close'])

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Ichimoku_Tenkan'] = visible_period['ITS_9']
        data['Ichimoku_Kijun'] = visible_period['IKS_26']
        data['Ichimoku_SenkouA'] = visible_period['ISA_9']
        data['Ichimoku_SenkouB'] = visible_period['ISB_26']
        data['Ichimoku_Chikou'] = visible_period['ICS_26']
        
        # Señales de Ichimoku
        data['Ichimoku_Signal'] = 0
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] > data['Ichimoku_Kijun']) & (data['Close'] > data['Ichimoku_SenkouA']) & (data['Close'] > data['Ichimoku_SenkouB']), 1, data['Ichimoku_Signal'])  # Señal de compra
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] < data['Ichimoku_Kijun']) & (data['Close'] < data['Ichimoku_SenkouA']) & (data['Close'] < data['Ichimoku_SenkouB']), -1, data['Ichimoku_Signal'])  # Señal de venta
        data['Position'] = data['Ichimoku_Signal']
        
    elif indicator == 'Ichimoku_Kumo':
        # Calcular Ichimoku Kinko Hyo usando pandas_ta
        visible_period, forward_period = ta.ichimoku(data['High'], data['Low'], data['Close'])

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Ichimoku_Tenkan'] = visible_period['ITS_9']
        data['Ichimoku_Kijun'] = visible_period['IKS_26']
        data['Ichimoku_SenkouA'] = visible_period['ISA_9']
        data['Ichimoku_SenkouB'] = visible_period['ISB_26']
        data['Ichimoku_Chikou'] = visible_period['ICS_26']
        
        # Señales de Ichimoku
        data['Ichimoku_Signal'] = 0
        data['Ichimoku_Signal'] = np.where((data['Close'] > data['Ichimoku_SenkouA']) & (data['Close'] > data['Ichimoku_SenkouB']), 1, data['Ichimoku_Signal'])  # Señal de compra
        data['Ichimoku_Signal'] = np.where((data['Close'] < data['Ichimoku_SenkouA']) & (data['Close'] > data['Ichimoku_SenkouB']), -1, data['Ichimoku_Signal'])  # Señal de venta
        data['Position'] = data['Ichimoku_Signal']
    
    elif indicator == 'Ichimoku_Chikou':
        # Calcular Ichimoku Kinko Hyo usando pandas_ta
        visible_period, forward_period = ta.ichimoku(data['High'], data['Low'], data['Close'])

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Ichimoku_Tenkan'] = visible_period['ITS_9']
        data['Ichimoku_Kijun'] = visible_period['IKS_26']
        data['Ichimoku_SenkouA'] = visible_period['ISA_9']
        data['Ichimoku_SenkouB'] = visible_period['ISB_26']
        data['Ichimoku_Chikou'] = visible_period['ICS_26']
        
        # Señales de Ichimoku
        data['Ichimoku_Signal'] = 0
        data['Ichimoku_Signal'] = np.where((data['Close'] > data['Ichimoku_SenkouA']) & (data['Close'] > data['Ichimoku_SenkouB']) & (data['Ichimoku_Chikou'] > data['Ichimoku_SenkouA']), 1, data['Ichimoku_Signal'])  # Señal de compra
        data['Ichimoku_Signal'] = np.where((data['Close'] < data['Ichimoku_SenkouA']) & (data['Close'] > data['Ichimoku_SenkouB']) & (data['Ichimoku_Chikou'] < data['Ichimoku_SenkouA']), -1, data['Ichimoku_Signal'])  # Señal de venta
        data['Position'] = data['Ichimoku_Signal']
    
    elif indicator == 'Ichimoku_Cruce':
        # Calcular Ichimoku Kinko Hyo usando pandas_ta
        visible_period, forward_period = ta.ichimoku(data['High'], data['Low'], data['Close'])

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Ichimoku_Tenkan'] = visible_period['ITS_9']
        data['Ichimoku_Kijun'] = visible_period['IKS_26']
        data['Ichimoku_SenkouA'] = visible_period['ISA_9']
        data['Ichimoku_SenkouB'] = visible_period['ISB_26']
        data['Ichimoku_Chikou'] = visible_period['ICS_26']
        
        # Señales de Ichimoku
        data['Ichimoku_Signal'] = 0
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] > data['Ichimoku_Kijun']), 1, data['Ichimoku_Signal'])  # Señal de compra
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] < data['Ichimoku_Kijun']), -1, data['Ichimoku_Signal'])  # Señal de venta
        data['Position'] = data['Ichimoku_Signal']

    elif indicator == 'Ichimoku_PSAR':
        # Calcular Ichimoku Kinko Hyo usando pandas_ta
        visible_period, forward_period = ta.ichimoku(data['High'], data['Low'], data['Close'])

        # Asignar las columnas al DataFrame 'data' según sea necesario
        data['Ichimoku_Tenkan'] = visible_period['ITS_9']
        data['Ichimoku_Kijun'] = visible_period['IKS_26']
        data['Ichimoku_SenkouA'] = visible_period['ISA_9']
        data['Ichimoku_SenkouB'] = visible_period['ISB_26']
        data['Ichimoku_Chikou'] = visible_period['ICS_26']

        psar_result = ta.psar(data['High'], data['Low'], data['Close'], af0=0.02, af=0.02, max_af=0.2)
        data['PSAR'] = psar_result['PSARr_0.02_0.2']
        
        # Señales de Ichimoku
        data['Ichimoku_Signal'] = 0
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] > data['Ichimoku_Kijun']) & (data['Close'] > data['PSAR']), 1, data['Ichimoku_Signal'])  # Señal de compra
        data['Ichimoku_Signal'] = np.where((data['Ichimoku_Tenkan'] < data['Ichimoku_Kijun']) & (data['Close'] < data['PSAR']), -1, data['Ichimoku_Signal'])  # Señal de venta
        data['Position'] = data['Ichimoku_Signal']
    
    else:
        raise ValueError("El parámetro 'indicator' debe ser 'SMA', 'EMA', 'RSI', 'MACD', 'Bollinger', 'Stochastic', 'Volume', 'EMA_RSI' o 'Ichimoku'.")
    
    return data


In [4]:
def simulate_backtesting(data, initial_capital, purchase_fraction, sell_fraction, position_column, take_profit_pct, stop_loss_pct):
    data['Portfolio Value'] = initial_capital
    data['Holdings'] = 0.0
    data['Cash'] = initial_capital
    data['Shares Owned'] = 0.0
    data['Take Profit'] = np.nan
    data['Stop Loss'] = np.nan

    for i in range(0, len(data)):
        if data[position_column][i] == 1:  # Comprar
            amount_to_spend = data['Cash'].iloc[i-1] * purchase_fraction
            shares_bought = amount_to_spend / data['Close'][i]
            data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1] + shares_bought
            data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1] - shares_bought * data['Close'][i]
            data.loc[data.index[i], 'Take Profit'] = data['Close'][i] * (1 + take_profit_pct)
            data.loc[data.index[i], 'Stop Loss'] = data['Close'][i] * (1 - stop_loss_pct)
        elif data[position_column][i] == -1:  # Vender
            if data['Close'][i] >= data['Take Profit'][i-1]:
                shares_to_sell = data['Holdings'].iloc[i-1] * sell_fraction
                data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1] + shares_to_sell * data['Close'][i]
                data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1] - shares_to_sell
            elif data['Close'][i] <= data['Stop Loss'][i-1]:
                shares_to_sell = data['Holdings'].iloc[i-1] * sell_fraction
                data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1] + shares_to_sell * data['Close'][i]
                data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1] - shares_to_sell
            else:
                data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1]
                data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1]
        else:  # Mantener
            data.loc[data.index[i], 'Holdings'] = data['Holdings'].iloc[i-1]
            data.loc[data.index[i], 'Cash'] = data['Cash'].iloc[i-1]
            data.loc[data.index[i], 'Take Profit'] = data['Take Profit'][i-1]
            data.loc[data.index[i], 'Stop Loss'] = data['Stop Loss'][i-1]

        data.loc[data.index[i], 'Portfolio Value'] = data['Cash'][i] + data['Holdings'][i] * data['Close'][i]
        data.loc[data.index[i], 'Shares Owned'] = data['Holdings'][i]

    final_portfolio_value = data['Portfolio Value'].iloc[-1]
    total_return = (final_portfolio_value - initial_capital) / initial_capital * 100
    return final_portfolio_value, total_return, data

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_results(data, indicator, ticker):
    # Asegurarse de que no hay NaN en las columnas que se van a plotear
    required_columns = ['Close', 'Portfolio Value', 'Shares Owned']
    
    # Añadir columnas específicas para cada indicador según las señales generadas
    if indicator in ['SMA', 'EMA']:
        required_columns += [f'{indicator}_50', f'{indicator}_200', 'Position']
    elif indicator == 'RSI':
        required_columns += ['RSI', 'Position']
    elif indicator == 'Bollinger':
        required_columns += ['Bollinger_Upper', 'Bollinger_Middle', 'Bollinger_Lower', 'Position']
    elif indicator == 'Stochastic':
        required_columns += ['Stoch_K', 'Stoch_D', 'Position']
    elif indicator == 'Volume':
        required_columns += ['Volume', 'Volume_Signal', 'Position']
    elif indicator == 'EMA_RSI':
        required_columns += ['EMA_50', 'EMA_200' 'RSI', 'Position']
    elif indicator == 'Ichimoku':
        required_columns += ['Ichimoku_Tenkan', 'Ichimoku_Kijun', 'Ichimoku_SenkouA', 'Ichimoku_SenkouB',
                             'Ichimoku_SenkouA', 'Ichimoku_Chikou', 'Position']
    
    # Eliminar filas con NaN en columnas requeridas
    data = data.dropna(subset=required_columns)

    # Visualización de resultados con Plotly
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        row_heights=[1, 1],
                        specs=[[{"secondary_y": True}]] * 2)  # Especificación para todos los subgráficos con secondary_y=True

    # Gráfico de precios y indicador seleccionado
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Precio de Cierre', line=dict(color='blue')), row=1, col=1, secondary_y=False)
    
    # Añadir gráficos específicos para cada indicador
    if indicator in ['SMA', 'EMA']:
        fig.add_trace(go.Scatter(x=data.index, y=data[f'{indicator}_50'], mode='lines', name=f'{indicator} de 50 días', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data[f'{indicator}_200'], mode='lines', name=f'{indicator} de 200 días', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'RSI':
        fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Bollinger':
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Upper'], mode='lines', name='Bollinger Upper', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Middle'], mode='lines', name='Bollinger Middle', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Bollinger_Lower'], mode='lines', name='Bollinger Lower', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Stochastic':
        fig.add_trace(go.Scatter(x=data.index, y=data['Stoch_K'], mode='lines', name='Stochastic K', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['Stoch_D'], mode='lines', name='Stochastic D', line=dict(color='purple')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'Volume':
        fig.add_trace(go.Scatter(x=data.index, y=data['Volume'], mode='lines', name='Volumen', line=dict(color='green')), row=1, col=1, secondary_y=True)
        
    elif indicator == 'EMA_RSI':
        fig.add_trace(go.Scatter(x=data.index, y=data['EMA_50'], mode='lines', name=f'EMA de 50 días', line=dict(color='orange')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['EMA_200'], mode='lines', name=f'EMA de 200 días', line=dict(color='green')), row=1, col=1, secondary_y=True)
        fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI', line=dict(color='purple')), row=1, col=1, secondary_y=True)

    elif indicator == 'Ichimoku':
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_Tenkan'], mode='lines', name='Tenkan-sen', line=dict(color='orange')), row=1, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_Kijun'], mode='lines', name='Kijun-sen', line=dict(color='green')), row=1, col=1, secondary_y=False)
        
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_SenkouA'], mode='lines', name='Senkou Span A', line=dict(color='green')), row=1, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_SenkouA'], mode='lines', name='Senkou Span A (ISA) - Relleno', fill='tonexty', line=dict(color='rgba(0,100,80,0.2)')), row=1, col=1)
        
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_SenkouB'], mode='lines', name='Senkou Span B', line=dict(color='red')), row=1, col=1, secondary_y=False)
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_SenkouB'], mode='lines', name='Senkou Span B (ISB) - Relleno', fill='tonexty', line=dict(color='rgba(100,0,0,0.2)')), row=1, col=1)
        
        fig.add_trace(go.Scatter(x=data.index, y=data['Ichimoku_Chikou'], mode='lines', name='Chikou Span', line=dict(color='purple')), row=1, col=1, secondary_y=False)

        
    # Señales de compra y venta para el indicador seleccionado
    if 'Position' in data.columns:
        if indicator in ['SMA', 'EMA']:
            fig.add_trace(go.Scatter(x=data[data['Position'] == 1].index, y=data[f'{indicator}_50'][data['Position'] == 1], mode='markers', marker_symbol='triangle-up', marker_color='green', marker_size=10, name=f'Señal de Compra ({indicator})'), row=1, col=1)
            fig.add_trace(go.Scatter(x=data[data['Position'] == -1].index, y=data[f'{indicator}_50'][data['Position'] == -1], mode='markers', marker_symbol='triangle-down', marker_color='red', marker_size=10, name=f'Señal de Venta ({indicator})'), row=1, col=1)
        else:
            # Para otros indicadores donde no se usa {indicator}_50
            fig.add_trace(go.Scatter(x=data[data['Position'] == 1].index, y=data['Close'][data['Position'] == 1], mode='markers', marker_symbol='triangle-up', marker_color='green', marker_size=10, name=f'Señal de Compra ({indicator})'), row=1, col=1)
            fig.add_trace(go.Scatter(x=data[data['Position'] == -1].index, y=data['Close'][data['Position'] == -1], mode='markers', marker_symbol='triangle-down', marker_color='red', marker_size=10, name=f'Señal de Venta ({indicator})'), row=1, col=1)

    # Gráfico del valor de la cartera y acciones poseídas para el indicador seleccionado
    fig.add_trace(go.Scatter(x=data.index, y=data['Portfolio Value'], mode='lines', name=f'Valor de la Cartera ({indicator})', line=dict(color='purple')), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=data.index, y=data['Shares Owned'], mode='lines', name=f'Acciones Poseídas ({indicator})', line=dict(color='black')), row=2, col=1, secondary_y=True)

    # Actualizar diseño del gráfico
    fig.update_layout(title=f'Estrategia de Trading de {ticker} con {indicator}', xaxis_title='Fecha', template='plotly_white')
    fig.update_yaxes(title_text='Precio', row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Indicador', row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text='Valor de la Cartera', row=2, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Acciones Poseídas', row=2, col=1, secondary_y=True)

    fig.show()


In [6]:
def backtest_strategy(start, end, interval, initial_capital, ticker, purchase_fraction, sell_fraction, indicator, mode, take_profit_pct, stop_loss_pct):
    # Descargar datos y calcular señales
    data = download_data(start, end, interval, ticker)
    data = calculate_signals(data.copy(), indicator=indicator)

    # Simulación
    final_value, total_return, data_backtesting = simulate_backtesting(data.copy(),
                                                                       initial_capital,
                                                                       purchase_fraction,
                                                                       sell_fraction,
                                                                       'Position',
                                                                       take_profit_pct,
                                                                       stop_loss_pct)
    if mode == 'plots':
        # Plotear resultados
        plot_results(data_backtesting, indicator, ticker)

    return final_value, total_return

In [41]:

'''
INDICADORES:
'SMA': Medias Móviles Simples.
'EMA': Medias Móviles Exponenciales.
'RSI': Índice de Fuerza Relativa.
'Bollinger': Bandas de Bollinger.
'Stochastic': Oscilador Estocástico.
'Volume': Volumen.
'EMA_RSI':
'Ichimoku'
'Ichimoku_Kumo'
'Ichimoku_Chikou'
'Ichimoku_Cruce'

INTERVALOS:
[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

MODOS:
plots: se muestran las gráficas de cada uno de los indicadores
table: solo se muestra la tabla final que recoge el resumen de las retabilidades
'''
# Valor
ticker = 'NVDA'
indicator='Hold'
mode = 'plots'

#Fechas
start= '2024-01-01'
end= '2024-06-14'
interval = '1h'

#Capital
initial_capital = 1000
purchase_fraction = 0.80  # 80% del capital para compras
sell_fraction = 0.60  # 60% del capital para ventas

#Trades
take_profit_pct = 0.5  # Porcentaje de beneficio objetivo (5%)
stop_loss_pct = 0.03  # Porcentaje de pérdida máxima admitida (3%)

final_value, total_return = backtest_strategy(start,
                                              end,
                                              interval,
                                              initial_capital,
                                              ticker,
                                              purchase_fraction,
                                              sell_fraction,
                                              indicator,
                                              mode,
                                              take_profit_pct,
                                              stop_loss_pct)

print(f'Valor final de la cartera ({indicator}): ${final_value:.2f}')
print(f'Retorno total ({indicator}): {total_return:.2f}%')

[*********************100%%**********************]  1 of 1 completed


In [35]:
'''
INDICADORES:
'SMA': Medias Móviles Simples.
'EMA': Medias Móviles Exponenciales.
'RSI': Índice de Fuerza Relativa.
'Bollinger': Bandas de Bollinger.
'Stochastic': Oscilador Estocástico.
'Volume': Volumen.
'EMA_RSI':

INTERVALOS:
[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

MODOS:
plots: se muestran las gráficas de cada uno de los indicadores
table: solo se muestra la tabla final que recoge el resumen de las retabilidades
'''

# Lista de todos los indicadores a probar
indicadores = ['Hold','SMA', 'EMA', 'RSI', 'Bollinger', 'Stochastic', 'Volume', 'EMA_RSI', 'Ichimoku','Ichimoku_Kumo','Ichimoku_Chikou','Ichimoku_Cruce','Ichimoku_PSAR']
resultados = []

# Parámetros comunes para el backtest
start= '2024-01-01'
end= '2024-06-14'
interval = '1h'
initial_capital = 1000
purchase_fraction = 0.80  # 80% del capital para compras
sell_fraction = 0.60  # 60% del capital para ventas
ticker = 'QQQ'
mode = 'table'

#Trades
take_profit_pct = 0.10  # Porcentaje de beneficio objetivo (5%)
stop_loss_pct = 0.03  # Porcentaje de pérdida máxima admitida (3%)

# Bucle para probar todos los indicadores
for indicator in indicadores:
    try:
        # Ejecutar el backtest para el indicador actual
        final_value, total_return = backtest_strategy(start,
                                                      end,
                                                      interval,
                                                      initial_capital,
                                                      ticker,
                                                      purchase_fraction,
                                                      sell_fraction,
                                                      indicator,
                                                      mode,
                                                      take_profit_pct,
                                                      stop_loss_pct)

        # Almacenar los resultados del backtest para el indicador actual
        resultados.append({
            'Indicador': indicator,
            'Valor Final de la Cartera': final_value,
            'Retorno Total': total_return
        })

        # Imprimir los resultados del backtest para el indicador actual
        print(f'Valor final de la cartera ({indicator}): ${final_value:.2f}')
        print(f'Retorno total ({indicator}): {total_return:.2f}%')
        print()  # Salto de línea para separar los resultados de diferentes indicadores

    except Exception as e:
        print(f'Ocurrió un error al procesar el indicador {indicator}: {str(e)}')
        print(f'Saltando al siguiente indicador...\n')

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

ticker_obj = yf.Ticker(ticker)

# Mostrar la tabla resumen de resultados
print(f"\nTabla Resumen de Rendimientos de {ticker_obj.info['longName']} por Indicador:")
print(f"Inicio de inversión: {start}")
print(f"Fin de inversión: {end}")
print(f"Temporalidad: {interval}\n")
print(df_resultados)


[*********************100%%**********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:

'''
INDICADORES:
'SMA': Medias Móviles Simples.
'EMA': Medias Móviles Exponenciales.
'RSI': Índice de Fuerza Relativa.
'Bollinger': Bandas de Bollinger.
'Stochastic': Oscilador Estocástico.
'Volume': Volumen.
'EMA_RSI':
'Ichimoku'
'Ichimoku_Kumo'
'Ichimoku_Chikou'
'Ichimoku_Cruce'

INTERVALOS:
[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

MODOS:
plots: se muestran las gráficas de cada uno de los indicadores
table: solo se muestra la tabla final que recoge el resumen de las retabilidades
'''
# Valor
ticker = 'NVDA'
indicator='Ichimoku_PSAR'
mode = 'plots'

#Fechas
start= '2024-01-01'
end= '2024-06-14'
interval = '1h'

#Capital
initial_capital = 1000
purchase_fraction = 0.80  # 80% del capital para compras
sell_fraction = 0.60  # 60% del capital para ventas

#Trades
take_profit_pct = 0.5  # Porcentaje de beneficio objetivo (5%)
stop_loss_pct = 0.03  # Porcentaje de pérdida máxima admitida (3%)

final_value, total_return = backtest_strategy(start,
                                              end,
                                              interval,
                                              initial_capital,
                                              ticker,
                                              purchase_fraction,
                                              sell_fraction,
                                              indicator,
                                              mode,
                                              take_profit_pct,
                                              stop_loss_pct)

print(f'Valor final de la cartera ({indicator}): ${final_value:.2f}')
print(f'Retorno total ({indicator}): {total_return:.2f}%')

[*********************100%%**********************]  1 of 1 completed


KeyError: 'PSARr_0.2_0.2'